# GeoMx Imaging Processing
Cole Nawrocki

In [1]:
from tifffile import TiffFile
from ome_types import from_xml
import geojson

## TMA1

In [ ]:
tma1_path = "/Users/ccn22/Library/CloudStorage/OneDrive-MassGeneralBrigham/geomx/wu-gu-prostate-bladder-data/images/Wu_Gu_TMA_1.ome.tiff"

In [ ]:
with TiffFile(tma1_path) as tif:
    ome_xml = tif.ome_metadata

ome = from_xml(ome_xml)

In [4]:
ome

OME(
   plates=[{'id': 'Plate:0', 'rows': 8, 'columns': 12}],
   screens=[{'reagents': [{'id': 'Reagent:0', 'reagent_identifier': 'Human NGS Whole Transcriptome Atlas RNA'}], 'plate_refs': [{'id': 'Plate:0'}], 'id': 'Screen:0'}],
   instruments=[{'microscope': {'manufacturer': 'Nanostring', 'model': 'GeoMx', 'kind': 'microscope'}, 'annotation_refs': [{'id': 'Annotation:0'}], 'id': 'Instrument:0'}],
   images=[<1 field_type>],
   structured_annotations={'xml_annotations': [{'id': 'Annotation:1', 'value': {'any_elements': [{'qname': '{http://www.openmicroscopy.org/Schemas/OME/2016-06}ChannelInfo', 'text': '', 'children': [{'qname': '{http://www.openmicroscopy.org/Schemas/OME/2016-06}Name', 'text': 'Blue'}, {'qname': '{http://www.openmicroscopy.org/Schemas/OME/2016-06}Dye', 'text': 'SYTO 13'}, {'qname': '{http://www.openmicroscopy.org/Schemas/OME/2016-06}DyeDisplayName', 'text': 'FITC'}, {'qname': '{http://www.openmicroscopy.org/Schemas/OME/2016-06}DyeWavelength', 'text': '525nm'}, {'qnam

In [6]:
ome.rois[2]

ROI(
   id='ROI:2',
   union={'labels': [{'stroke_color': Color('white', rgb=(255, 255, 255)), 'text': '003', 'font_family': <Shape_FontFamily.SANS_SERIF: 'sans-serif'>, 'font_size': 64, 'font_style': <Shape_FontStyle.NORMAL: 'Normal'>, 'id': 'Shape:6', 'x': 17235.0, 'y': 21626.0, 'kind': 'label'}], 'masks': [{'fill_color': Color('#c0e0c080', rgb=(192, 224, 192, 0.5019607843137255)), 'text': 'Full ROI', 'id': 'Shape:8', 'bin_data': {'value': b'\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff

In [7]:
def rectangle_to_polygon(rect):
    # Get rectangle properties
    x = rect.x
    y = rect.y
    w = rect.width
    h = rect.height
    # Define the four corners (closed ring)
    return [
        [x, y],
        [x + w, y],
        [x + w, y + h],
        [x, y + h],
        [x, y]  # Close the polygon
    ]

def polygon_to_points(polygon):
    # Convert OME polygon object to list of [x, y] points
    point_list = polygon.points.split(" ")
    split_list = [s.split(',') for s in point_list]
    float_list = [[float(item) for item in sublist] for sublist in split_list]
    final_list = float_list.append(float_list[0])  # Close the polygon
    return split_list

In [37]:
roi_list = []
for roi in ome.rois:
    roi_name = roi.union.labels[0].text
    if len(roi.union.polygons) == 0:
        rect = roi.union.rectangles[0]
        final_polygon = rectangle_to_polygon(rect)
    else:
        final_polygon = polygon_to_points(roi.union.polygons[0])
    feature = {
        "type": "Feature",
        "properties": {"id": "tma1_" + roi_name, "name": roi_name},
        "geometry": {
            "type": "Polygon",
            "coordinates": [final_polygon]
        }
    }
    roi_list.append(feature)
    

In [38]:
roi_list

[{'type': 'Feature',
  'properties': {'id': 'tma1_001', 'name': '001'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[29652.820048150228, 16687.470862243263],
     [31154.84380861342, 16687.470862243263],
     [31154.84380861342, 18179.638568479688],
     [29652.820048150228, 18179.638568479688],
     [29652.820048150228, 16687.470862243263]]]}},
 {'type': 'Feature',
  'properties': {'id': 'tma1_002', 'name': '002'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[27383.57664365008, 18591.432043417037],
     [28881.10706396231, 18591.432043417037],
     [28881.10706396231, 19666.23291511205],
     [27383.57664365008, 19666.23291511205],
     [27383.57664365008, 18591.432043417037]]]}},
 {'type': 'Feature',
  'properties': {'id': 'tma1_003', 'name': '003'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[16534.602657591044, 21722.307606052786],
     [18050.855328532285, 21722.307606052786],
     [18050.855328532285, 22734.340907064823],
     [16534.602657591044, 2

In [39]:
geojson_output = geojson.FeatureCollection(roi_list)

In [40]:
geojson_output

{"features": [{"geometry": {"coordinates": [[[29652.820048150228, 16687.470862243263], [31154.84380861342, 16687.470862243263], [31154.84380861342, 18179.638568479688], [29652.820048150228, 18179.638568479688], [29652.820048150228, 16687.470862243263]]], "type": "Polygon"}, "properties": {"id": "tma1_001", "name": "001"}, "type": "Feature"}, {"geometry": {"coordinates": [[[27383.57664365008, 18591.432043417037], [28881.10706396231, 18591.432043417037], [28881.10706396231, 19666.23291511205], [27383.57664365008, 19666.23291511205], [27383.57664365008, 18591.432043417037]]], "type": "Polygon"}, "properties": {"id": "tma1_002", "name": "002"}, "type": "Feature"}, {"geometry": {"coordinates": [[[16534.602657591044, 21722.307606052786], [18050.855328532285, 21722.307606052786], [18050.855328532285, 22734.340907064823], [16534.602657591044, 22734.340907064823], [16534.602657591044, 21722.307606052786]]], "type": "Polygon"}, "properties": {"id": "tma1_003", "name": "003"}, "type": "Feature"},

In [41]:
for feature in geojson_output["features"]:
    coords = feature["geometry"]["coordinates"][0]
    # If not already closed, close it
    if coords[0] != coords[-1]:
        coords.append(coords[0])
    # Re-assign to make sure the geometry is updated
    feature["geometry"]["coordinates"] = [coords]

In [42]:
with open("tma1_roi_shapes.geojson", "w") as f:
    geojson.dump(roi_list, f, indent=2)

## TMA2

In [44]:
tma2_path = "/Users/ccn22/Library/CloudStorage/OneDrive-MassGeneralBrigham/geomx/wu-gu-prostate-bladder-data/images/Wu_Gu_TMA_2.ome.tiff"
with TiffFile(tma2_path) as tif:
    ome_xml = tif.ome_metadata

ome = from_xml(ome_xml)

roi_list = []
for roi in ome.rois:
    roi_name = roi.union.labels[0].text
    if len(roi.union.polygons) == 0:
        rect = roi.union.rectangles[0]
        final_polygon = rectangle_to_polygon(rect)
    else:
        final_polygon = polygon_to_points(roi.union.polygons[0])
    feature = {
        "type": "Feature",
        "properties": {"id": "tma2_" + roi_name, "name": roi_name},
        "geometry": {
            "type": "Polygon",
            "coordinates": [final_polygon]
        }
    }
    roi_list.append(feature)

    geojson_output = geojson.FeatureCollection(roi_list)

for feature in geojson_output["features"]:
    coords = feature["geometry"]["coordinates"][0]
    # If not already closed, close it
    if coords[0] != coords[-1]:
        coords.append(coords[0])
    # Re-assign to make sure the geometry is updated
    feature["geometry"]["coordinates"] = [coords]

with open("tma2_roi_shapes.geojson", "w") as f:
    geojson.dump(roi_list, f, indent=2)